# Data Science Collaboration Project - Analysis Notebook

This notebook demonstrates the complete data science workflow using our collaborative project structure.

## Table of Contents
1. [Environment Setup](#environment-setup)
2. [Data Loading and Exploration](#data-loading-and-exploration)
3. [Data Preprocessing](#data-preprocessing)
4. [Exploratory Data Analysis](#exploratory-data-analysis)
5. [Model Training and Evaluation](#model-training-and-evaluation)
6. [Results and Conclusions](#results-and-conclusions)

---


## 1. Environment Setup

First, let's import all necessary libraries and set up our environment.


In [ ]:
# Standard library imports
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path for importing our modules
sys.path.append('../src')

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Our custom modules
from data_preprocessing import preprocess_pipeline, load_raw_data, clean_missing_values
from model_training import ModelTrainer
from utils import setup_plotting_style, check_data_quality, create_data_profile

# Set up plotting style
setup_plotting_style()

print("Environment setup complete!")
print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 2. Data Loading and Exploration

Since we don't have real data yet, let's create a sample dataset to demonstrate our workflow.


In [ ]:
# Create sample dataset for demonstration
np.random.seed(42)

# Generate synthetic customer data
n_samples = 1000

data = {
    'customer_id': range(1, n_samples + 1),
    'age': np.random.normal(35, 12, n_samples).astype(int),
    'income': np.random.normal(50000, 15000, n_samples),
    'spending_score': np.random.normal(50, 25, n_samples),
    'years_as_customer': np.random.exponential(3, n_samples),
    'num_purchases': np.random.poisson(12, n_samples),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_samples),
    'gender': np.random.choice(['M', 'F'], n_samples),
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples, p=[0.3, 0.4, 0.2, 0.1])
}

# Create target variable (high-value customer)
# Based on income, spending_score, and num_purchases
high_value = ((data['income'] > 55000) & 
              (data['spending_score'] > 60) & 
              (data['num_purchases'] > 15)).astype(int)

data['high_value_customer'] = high_value

# Create DataFrame
df = pd.DataFrame(data)

# Introduce some missing values for demonstration
missing_indices = np.random.choice(df.index, size=50, replace=False)
df.loc[missing_indices, 'income'] = np.nan

missing_indices = np.random.choice(df.index, size=30, replace=False)
df.loc[missing_indices, 'spending_score'] = np.nan

print("Sample dataset created!")
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
df.head()


In [ ]:
# Basic dataset information
print("Dataset Info:")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nData types:")
print(df.dtypes)
print(f"\nBasic statistics:")
df.describe()


## 3. Data Quality Assessment

Let's use our custom utility functions to assess data quality.


In [ ]:
# Perform data quality check using our custom function
quality_report = check_data_quality(df)

print("Data Quality Report Summary:")
print("=" * 50)
print(f"Total missing values: {sum(quality_report['missing_values']['total_missing'].values())}")
print(f"Duplicate rows: {quality_report['duplicates']['duplicate_rows']}")
print(f"Potential issues found: {len(quality_report['potential_issues'])}")

if quality_report['potential_issues']:
    print("\nPotential Issues:")
    for issue in quality_report['potential_issues']:
        print(f"- {issue}")
else:
    print("No potential issues detected!")


In [ ]:
# Create data profile visualization
create_data_profile(df, "Customer Dataset Profile")


## 4. Exploratory Data Analysis

Let's explore the relationships in our data and understand the patterns.


In [ ]:
# Distribution of target variable
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
df['high_value_customer'].value_counts().plot(kind='bar', color=['lightcoral', 'skyblue'])
plt.title('Distribution of High-Value Customers')
plt.xlabel('High Value Customer')
plt.ylabel('Count')
plt.xticks([0, 1], ['No', 'Yes'], rotation=0)

plt.subplot(1, 2, 2)
target_pct = df['high_value_customer'].value_counts(normalize=True) * 100
plt.pie(target_pct.values, labels=['No', 'Yes'], autopct='%1.1f%%', colors=['lightcoral', 'skyblue'])
plt.title('Percentage of High-Value Customers')

plt.tight_layout()
plt.show()

print(f"High-value customers: {df['high_value_customer'].sum()} ({df['high_value_customer'].mean():.1%})")


In [ ]:
# Correlation analysis
from utils import correlation_analysis

print("Performing correlation analysis...")
high_corr_pairs = correlation_analysis(df, threshold=0.3, plot=True)

if len(high_corr_pairs) > 0:
    print("\nHighly correlated feature pairs:")
    print(high_corr_pairs.round(3))
else:
    print("No highly correlated features found.")


In [ ]:
# Feature distributions by target variable
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distributions by High-Value Customer Status', fontsize=16)

# Age distribution
axes[0, 0].hist(df[df['high_value_customer'] == 0]['age'], alpha=0.7, label='Not High-Value', bins=20)
axes[0, 0].hist(df[df['high_value_customer'] == 1]['age'], alpha=0.7, label='High-Value', bins=20)
axes[0, 0].set_title('Age Distribution')
axes[0, 0].set_xlabel('Age')
axes[0, 0].legend()

# Income distribution
axes[0, 1].hist(df[df['high_value_customer'] == 0]['income'].dropna(), alpha=0.7, label='Not High-Value', bins=20)
axes[0, 1].hist(df[df['high_value_customer'] == 1]['income'].dropna(), alpha=0.7, label='High-Value', bins=20)
axes[0, 1].set_title('Income Distribution')
axes[0, 1].set_xlabel('Income')
axes[0, 1].legend()

# Spending Score distribution
axes[1, 0].hist(df[df['high_value_customer'] == 0]['spending_score'].dropna(), alpha=0.7, label='Not High-Value', bins=20)
axes[1, 0].hist(df[df['high_value_customer'] == 1]['spending_score'].dropna(), alpha=0.7, label='High-Value', bins=20)
axes[1, 0].set_title('Spending Score Distribution')
axes[1, 0].set_xlabel('Spending Score')
axes[1, 0].legend()

# Number of purchases distribution
axes[1, 1].hist(df[df['high_value_customer'] == 0]['num_purchases'], alpha=0.7, label='Not High-Value', bins=20)
axes[1, 1].hist(df[df['high_value_customer'] == 1]['num_purchases'], alpha=0.7, label='High-Value', bins=20)
axes[1, 1].set_title('Number of Purchases Distribution')
axes[1, 1].set_xlabel('Number of Purchases')
axes[1, 1].legend()

plt.tight_layout()
plt.show()


## 5. Model Training and Evaluation

Now let's use our custom model training module to build and evaluate machine learning models.


In [ ]:
# Prepare data for modeling
# First, handle missing values
df_model = df.copy()

# Fill missing values with median
df_model['income'] = df_model['income'].fillna(df_model['income'].median())
df_model['spending_score'] = df_model['spending_score'].fillna(df_model['spending_score'].median())

# Prepare features and target
X = df_model.drop(['customer_id', 'high_value_customer'], axis=1)
y = df_model['high_value_customer']

# Convert categorical variables to dummy variables
X = pd.get_dummies(X, drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Feature columns: {list(X.columns)}")


In [ ]:
# Initialize model trainer
trainer = ModelTrainer()

# Train multiple models for comparison
print("Training multiple models...")
models = {}

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
models['RandomForest'] = rf_model

# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)
models['LogisticRegression'] = lr_model

# Evaluate models
print("\nModel Evaluation Results:")
print("=" * 50)

for name, model in models.items():
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n{name}:")
    print(f"Accuracy: {accuracy:.4f}")
    
    # Detailed classification report
    print(f"\nClassification Report for {name}:")
    print(classification_report(y_test, y_pred))


In [ ]:
# Feature importance analysis (for Random Forest)
if hasattr(rf_model, 'feature_importances_'):
    # Create feature importance DataFrame
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
    plt.title('Top 10 Feature Importances (Random Forest)')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print("Top 10 Most Important Features:")
    print(feature_importance.head(10).round(4))


In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Create confusion matrices for both models
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for idx, (name, model) in enumerate(models.items()):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(f'Confusion Matrix - {name}')
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')

plt.tight_layout()
plt.show()


## 6. Results and Conclusions

### Key Findings

1. **Data Quality**: Our synthetic dataset had minimal quality issues, with only some missing values in income and spending score columns.

2. **Feature Importance**: The most important features for predicting high-value customers are:
   - Income levels
   - Spending scores
   - Number of purchases
   - Years as customer

3. **Model Performance**: Both Random Forest and Logistic Regression performed well, with Random Forest showing slightly better performance.

### Recommendations

1. **Data Collection**: Focus on collecting complete income and spending behavior data as these are strong predictors.

2. **Model Deployment**: The Random Forest model can be deployed for identifying high-value customers.

3. **Feature Engineering**: Consider creating interaction features between income, spending score, and purchase behavior.

4. **Monitoring**: Set up regular model retraining as customer behavior patterns may change over time.

### Next Steps

1. **Hyperparameter Tuning**: Optimize model parameters using grid search or random search.
2. **Cross-Validation**: Implement more robust cross-validation strategies.
3. **Model Interpretability**: Use SHAP or LIME for better model interpretability.
4. **A/B Testing**: Set up A/B tests to validate model performance in production.


In [ ]:
# Save the best model for future use
import joblib
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the Random Forest model
model_path = '../models/high_value_customer_model.joblib'
joblib.dump(rf_model, model_path)

print(f"Model saved to: {model_path}")
print("Analysis complete!")
